# langchain 读写 neo4j


In [2]:
from langchain_neo4j import Neo4jGraph

from conf.config import *

neo4j_graph = Neo4jGraph(
    url=NEO4J_CONFIG['url'],
    username=NEO4J_CONFIG['user'],
    password=NEO4J_CONFIG['password'],
    database=NEO4J_CONFIG.get('database', 'neo4j'),
)


In [3]:
# 统计当前图中的节点和关系数量
neo4j_graph.query("""
MATCH (n)
WITH count(n) AS nodes
MATCH ()-[r]-()
RETURN nodes, count(r) AS relationships
""")


[{'nodes': 1533, 'relationships': 3286}]

In [ ]:
# 预览 Category1 -> Category2 -> Category3 的层级链路
neo4j_graph.query("""
MATCH (c1:Category1)-[:Belong]-(c2:Category2)-[:Belong]-(c3:Category3)
RETURN c1.name AS c1, collect(DISTINCT c2.name)[0..5] AS c2_list, collect(DISTINCT c3.name)[0..5] AS c3_list
LIMIT 5
""")


In [ ]:
# 按 SPU id 查询它的销售属性和值
neo4j_graph.query("""
MATCH (spu:SPU {id: $spu_id})-[:Have]->(sale_attr:SaleAttr)-[:Have]->(value:SaleAttrValue)
RETURN spu.id AS spu_id, spu.name AS spu_name, collect(DISTINCT sale_attr.name) AS sale_attrs, collect(DISTINCT value.name) AS sale_attr_values
LIMIT 10
""", params={'spu_id': 1})


In [ ]:
# 按品牌名称查 SKU 及其基础属性值
neo4j_graph.query("""
MATCH (brand:BaseTrademark {name: $brand})<-[:Belong]-(sku:SKU)
OPTIONAL MATCH (sku)-[:Have]->(val:BaseAttrValue)
RETURN brand.name AS brand, sku.id AS sku_id, collect(DISTINCT val.name) AS base_attr_values
LIMIT 10
""", params={'brand': '示例品牌'})
